In [6]:
# PymuPDFライブラリインストール(import fitz)
!pip install pymupdf

In [7]:
import pandas as pd
import numpy as np
import extract_images
import extract_figure

In [8]:
def merge_image_and_figure_info(input_folder, image_csv_filename, output_root_folder, pattern, figure_csv_filename, output_csv):
    """
    画像情報と図番号情報をマージしてCSVファイルに保存する関数。

    Args:
        input_folder (str): PDFファイルが格納されているフォルダのパス。
        image_csv_filename (str): 画像情報出力CSVファイルのパス。
        output_root_folder (str): 画像を保存するルートフォルダへのパス。
        pattern (str): 図番号の正規表現パターン。
        figure_csv_filename (str): 出力CSVファイルのパス。
        output_csv (str): 出力CSVファイルのパス。
    """
    # 画像情報と図番号情報を含むデータフレームを作成する
    image_info = extract_images.extract_images(input_folder, image_csv_filename, output_root_folder) # 画像情報を取得する関数
    figure_info = extract_figure.extract_figure(input_folder, pattern, figure_csv_filename)  # 図番号情報を取得する関数

    # マージしたデータを格納するリストを初期化
    merged_data = []

    # 画像の座標を取得
    for _, img_row in image_info.iterrows():
        img_page = img_row['ページ番号']
        img_x0, img_y0, img_x1, img_y1 = img_row['x0'], img_row['y0'], img_row['x1'], img_row['y1']

        # PDFファイルが一致する図番号情報をフィルタリング
        matching_figures = figure_info[(figure_info['PDFファイル'] == img_row['PDFファイル'])]

        if not matching_figures.empty:
            best_match = None
            min_distance = np.inf

            # 図番号座標を取得
            for _, fig_row in matching_figures.iterrows():
                fig_x0, fig_y0, fig_x1, fig_y1 = fig_row['x0'], fig_row['y0'], fig_row['x1'], fig_row['y1']

                # 画像と図番号の座標の差を計算(ユークリッド距離)
                distance = np.sqrt((img_x0 - fig_x0)**2 + (img_y0 - fig_y0)**2)
                

                # 座標の差が小さい方を選択
                if distance < min_distance:
                    best_match = fig_row
                    min_distance = distance

            if best_match is not None:
                # 図番号と画像の座標差をデータフレームに保存
                x0_diff = img_x0 - best_match['x0']
                y0_diff = img_y0 - best_match['y0']
                x1_diff = img_x1 - best_match['x1']
                y1_diff = img_y1 - best_match['y1']

                merged_data.append({
                    'PDFファイル': img_row['PDFファイル'],
                    'ページ番号': img_page,
                    '画像ファイル': img_row['画像ファイル'],
                    '図番号': best_match['図番号'],
                    'x0_diff': f'{x0_diff:.5f}',
                    'y0_diff': f'{y0_diff:.5f}',
                    'x1_diff': f'{x1_diff:.5f}',
                    'y1_diff': f'{y1_diff:.5f}'
                })

    # マージしたデータをデータフレームに変換
    merged_df = pd.DataFrame(merged_data)

    # マージしたデータフレームを保存
    merged_df.to_csv(output_csv, index=False)


In [9]:
# 使用例:
if __name__ == "__main__":
    input_folder = '../sample-pdf'  # PDFファイルが格納されているフォルダのパス
    image_csv_filename = 'image_info.csv' # 画像情報出力CSVファイルのパス。
    output_root_folder = "output-images" # 画像を保存するルートフォルダへのパス。
    pattern = r"【図 [0-9]+\-[0-9]+】"  # 図番号の正規表現パターン
    figure_csv_filename = 'figure_info.csv' # 図番号情報出力CSVファイルのパス。
    output_csv = 'merged_info.csv'  # 出力CSVファイルのパス
    merge_image_and_figure_info(input_folder, image_csv_filename, output_root_folder, pattern, figure_csv_filename, output_csv)


Matched text: 【図 1-1】 at coordinates: (84.95994567871094, 339.4860534667969), (91.95994567871094, 351.6526794433594)
Matched text: 【図 1-2】 at coordinates: (84.95994567871094, 1124.7660522460938), (91.95994567871094, 1136.9326782226562)
Matched text: 【図 1-3】 at coordinates: (84.95994567871094, 1853.166015625), (91.95994567871094, 1865.3326568603516)
Matched text: 【図 1-4】 at coordinates: (84.95994567871094, 2865.2460021972656), (91.95994567871094, 2877.412628173828)
Matched text: 【図 1-5】 at coordinates: (84.95994567871094, 3095.166015625), (91.95994567871094, 3107.3327026367188)
Matched text: 【図 1-6】 at coordinates: (84.95994567871094, 3736.2059936523438), (91.95994567871094, 3748.3726196289062)
Matched text: 【図 1-7】 at coordinates: (84.95994567871094, 4397.885971069336), (91.95994567871094, 4410.0526123046875)
Matched text: 【図 1-8】 at coordinates: (84.95994567871094, 5438.765960693359), (91.95994567871094, 5450.932586669922)
Matched text: 【図 2-1】 at coordinates: (84.95994567871094, 259.